In [11]:
import requests
import os
from bs4 import BeautifulSoup
import re

import urllib.parse

from selenium import webdriver
import selenium.webdriver.support.ui as ui
from base64 import b64encode
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import time

In [12]:
siteUrl = "http://www.commercialtrucktrader.com/Medium-Duty-Trucks-For-Sale/search-results"
siteUrl = "?".join([siteUrl, "type=medium"])
print(siteUrl)
max_page = 0
seen_urls_array = []
proxy = {'host': '23.19.51.245', 'port': '40791', 'usr': '<<USERID>>', 'pwd': '<<PASSWORD>>'}

http://www.commercialtrucktrader.com/Medium-Duty-Trucks-For-Sale/search-results?type=medium


In [13]:
#<a href="/Medium-Duty-Trucks-For-Sale/search-results?type=medium&amp;page=1081" class="listings-pag-default">1081</a>

# Retrieve the page with tag results and set it up to be scraped
sitePage = requests.get(url=siteUrl)
sitePageSoup = BeautifulSoup(sitePage.content, 'lxml')

css = 'div.listings-pag-bottom > div > a.listings-pag-default'
pageLinks = sitePageSoup.select(css)
if len(pageLinks):
    max_page = int(pageLinks[-1]["href"].split('=')[2])
print(max_page)

1081


In [14]:
# Retrieve the page with tag results and set it up to be scraped
fp = webdriver.FirefoxProfile()
fp.set_preference('network.proxy.type', 1)
fp.set_preference('network.proxy.http', proxy['host'])
fp.set_preference('network.proxy.http_port', int(proxy['port']))
fp.set_preference('network.proxy.no_proxies_on', 'localhost, 127.0.0.1')
fp.add_extension('closeproxy.xpi')
credentials = '{usr}:{pwd}'.format(**proxy)
credentials = b64encode(credentials.encode('ascii')).decode('utf-8')
fp.set_preference('extensions.closeproxyauth.authtoken', credentials)

#set some privacy settings
fp.set_preference( "places.history.enabled", False )
fp.set_preference( "privacy.clearOnShutdown.offlineApps", True )
fp.set_preference( "privacy.clearOnShutdown.passwords", True )
fp.set_preference( "privacy.clearOnShutdown.siteSettings", True )
fp.set_preference( "privacy.sanitize.sanitizeOnShutdown", True )
fp.set_preference( "signon.rememberSignons", False )
fp.set_preference( "network.cookie.lifetimePolicy", 2 )
fp.set_preference( "network.dns.disablePrefetch", True )
fp.set_preference( "network.http.sendRefererHeader", 0 )

#if you're really hardcore about your security
#js can be used to reveal your true i.p.
fp.set_preference( "javascript.enabled", False )

#get a huge speed increase by not downloading images
fp.set_preference( "permissions.default.image", 2 )

driver = webdriver.Firefox(fp)

In [15]:
# set timeout information
driver.set_page_load_timeout(5)

finished = 0
while finished == 0:
    try:
        driver.get(siteUrl)
        finished = 1
    except Exception as e:
        print(str(e))
        time.sleep(5)

Message: Error loading page, timed out (checkLoad)



In [ ]:
wait = WebDriverWait(driver, 10)
pattern = re.compile('\d+')

while True:
    try:
        element = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, 'Next')))
        this_title = driver.find_element_by_tag_name('title').text
        match = pattern.search(this_title)
        page_number = int(match.group())
        #print(page_number)

        # Retrieve the page with tag results and set it up to be scraped
        tagPageSoup = BeautifulSoup(driver.page_source, 'lxml')

        pageLinks = tagPageSoup.select('div.listing-header > a > h2')
        for pageLink in pageLinks:
            pageUrl = urllib.parse.urljoin(siteUrl, re.sub(' ', '%20', pageLink.parent['href']))
            #print(pageUrl)
            if pageUrl not in seen_urls_array:
                seen_urls_array.append(pageUrl)
        
        element.click()
    except TimeoutException:
        break  # cannot click the button anymore
    except Exception as e:
        print(str(e))

    # TODO: wait for the results of the click action
    try:
        WebDriverWait(driver, 3).until(
            EC.title_contains('Medium Duty Trucks For Sale - Page ' + str(page_number+1))
        )
    except Exception as e:
        print(str(e))
driver.quit()

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 



In [ ]:
print(seen_urls_array)

In [7]:
bigFile = open('commercialtrucktraderMediumDutyTrucks.txt', 'a')
for pageUrl in seen_urls_array:
    page = requests.get(url=pageUrl)
    soup = BeautifulSoup(page.content, 'lxml')

    dealerLink = soup.select("body > div.details-container > div.details > div.details-default > div.details-right > strong")
    dealerName = dealerLink[0].get_text().strip()

    if dealerName == "Private Seller":
        model = soup.select("div.details > h1")
        if(len(model)):
            model = model[0].get_text().strip()
        else:
            model = "Unknown"
        price = soup.select("div.details-right > h2.lfloat")
        if(len(price)):
            price = price[0].get_text().strip()
        else:
            price = "Unknown"
        phone = soup.select('span.flipphone.bold.font1-1')
        if(len(phone)>=2):
            phone = phone[1].get_text().strip()
        else:
            phone = ""
        if len(phone):
            bigFile.write(model + '\t' + price + '\t' + phone + '\n')
bigFile.close()